In [9]:
import os
import pandas as pd
import numpy as np
from tensorflow.python import pywrap_tensorflow
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

In [10]:
def show_word_and_doc_embeddings():
    for tensor_name in var_to_shape_map:
        print("tensor_name: ", tensor_name) # get the variable name
        print(reader.get_tensor(tensor_name)) # get the variable
    
cwd=os.getcwd()
models_folder_name = os.path.join(cwd,'models')
path_to_preprocessed_texts=os.path.join(cwd,'texts','preprocessed_texts_for_doc2vec.pkl')
path_to_saved_model = os.path.join(models_folder_name,'doc2vec_recipes_checkpoint_v2_2.ckpt')

reader = pywrap_tensorflow.NewCheckpointReader(path_to_saved_model)
var_to_shape_map = reader.get_variable_to_shape_map() # 'var_to_shape_map' is a dictionary contains every tensor in the model
show_word_and_doc_embeddings()

doc_embeddings = reader.get_tensor('doc_embeddings')
doc_embeddings = doc_embeddings.tolist() 
    
df_preprocessed_texts = pd.read_pickle(path_to_preprocessed_texts)

tensor_name:  doc_embeddings
[[-2.95451665e+00  4.59613949e-01 -7.34912157e-01  2.37734243e-01
  -2.27572608e+00 -1.13571775e+00 -2.19216561e+00  7.18121827e-01]
 [ 1.62188962e-01  2.54911590e+00 -4.25274897e+00 -3.36331940e+00
  -3.25274616e-01  4.51885045e-01  1.72038853e+00  1.63924062e+00]
 [-2.48911858e+00 -1.50700307e+00 -3.78769898e+00 -1.61020786e-01
  -5.88994145e-01  3.37696910e-01  1.94820857e+00  8.88742328e-01]
 [-3.32748532e+00  1.73464191e+00 -1.14303243e+00  4.06971842e-01
  -3.18238318e-01 -1.79555088e-01  2.74744201e+00  9.10472631e-01]
 [-1.14692616e+00 -7.75474980e-02 -2.15224767e+00 -2.30233693e+00
  -2.53352094e+00 -1.94668162e+00  2.46474433e+00 -2.75559521e+00]
 [-1.02211803e-01 -1.08980262e+00 -2.63696289e+00  9.30477440e-01
  -1.00048080e-01 -1.95770085e+00  6.82825744e-01  1.30770016e+00]
 [-2.74998403e+00  1.14806545e+00 -1.60485291e+00 -1.03205509e-01
  -2.11798453e+00 -2.18712783e+00  4.11362314e+00 -1.25437415e+00]
 [ 2.29790583e-01  2.15259004e+00 -1.447

In [11]:
 dict_doc_embeddings = {
    'text_names': df_preprocessed_texts.text_names,
    'labels' : df_preprocessed_texts.labels,
    'doc_embeddings' : doc_embeddings
}
    

df_doc_embeddings=pd.DataFrame(dict_doc_embeddings)

In [12]:
doc_embeddings=df_doc_embeddings.doc_embeddings.values.tolist()
labels=df_doc_embeddings.labels.values.tolist()
text_names=df_doc_embeddings.text_names.values.tolist()

unique_labels=sorted(set(labels))
number_of_recipe_categories=len(unique_labels)
possible_category_indices=np.linspace(0,number_of_recipe_categories-1,number_of_recipe_categories,dtype=int)
labels2integers=dict(zip(unique_labels,possible_category_indices))

print(labels2integers)

category_indices=[]
for l in labels:
    category_index=labels2integers[l]
    category_indices.append(category_index)

{'tiramisu': 3, 'sushi': 2, 'steak': 1, 'sashimi': 0}


In [13]:
#PCA

pca = PCA(n_components=2)
pca_2D_embeddings = pca.fit_transform(doc_embeddings)

dict_PCA_2D = {
    'pca-1' : pca_2D_embeddings[:,0],
    'pca-2' : pca_2D_embeddings[:,1],
    'labels' : labels,
    'category_indices': category_indices,
    'text_names': text_names
}

df_PCA_2D=pd.DataFrame(dict_PCA_2D)

In [14]:
traces=[]

for n in range(0,number_of_recipe_categories):
    df_category=df_PCA_2D[(df_PCA_2D.category_indices==n)]
    point_name=str(df_category.labels[df_category.index[0]])
    trace = go.Scatter(
    x=df_category['pca-1'],
    y=df_category['pca-2'],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category['text_names'],
    )

    traces.append(trace)

layout = go.Layout(
    title='PCA reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [19]:
#t-SNE 2D
    
tsne2D = TSNE(perplexity=35, n_components=2, verbose=2, init='pca', n_iter=2000,
                  n_iter_without_progress=500, learning_rate=5,method='exact')
tsne_2D_embeddings = tsne2D.fit_transform(doc_embeddings)
    
dict_TSNE_2D = {
    'pca-1' : tsne_2D_embeddings[:,0],
    'pca-2' : tsne_2D_embeddings[:,1],
    'labels' : labels,
    'category_indices': category_indices,
    'text_names': text_names
}

df_TSNE_2D=pd.DataFrame(dict_TSNE_2D)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 45 / 45
[t-SNE] Mean sigma: 4.826525
[t-SNE] Iteration 50: error = 32.3205264, gradient norm = 0.4582085 (50 iterations in 0.060s)
[t-SNE] Iteration 100: error = 32.3266381, gradient norm = 0.4334671 (50 iterations in 0.048s)
[t-SNE] Iteration 150: error = 32.3276317, gradient norm = 0.4571517 (50 iterations in 0.028s)
[t-SNE] Iteration 200: error = 32.3304794, gradient norm = 0.4211170 (50 iterations in 0.026s)
[t-SNE] Iteration 250: error = 32.3302376, gradient norm = 0.4206915 (50 iterations in 0.027s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 32.330238
[t-SNE] Iteration 300: error = 0.0649177, gradient norm = 0.0081881 (50 iterations in 0.033s)
[t-SNE] Iteration 350: error = 0.0623780, gradient norm = 0.0001023 (50 iterations in 0.026s)
[t-SNE] Iteration 400: error = 0.0623774, gradient norm = 0.0000017 (50 iterations in 0.026s)
[t-SNE] Iteration 450: error = 0.06

In [20]:
traces=[]

for n in range(0,number_of_recipe_categories):
    df_category=df_TSNE_2D[(df_TSNE_2D.category_indices==n)]
    point_name=str(df_category.labels[df_category.index[0]])
    trace = go.Scatter(
    x=df_category['pca-1'],
    y=df_category['pca-2'],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category['text_names'],
    )

    traces.append(trace)

layout = go.Layout(
    title='t-SNE reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)